In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.2. Smoking Status Prediction/Output')
cur_date = "080422"

library(tidyverse)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)
library(pROC)
library(randomForest)

#reading in file
molecular_feature_df = read_csv("Input/20220217_mf_qrilc_w40.csv") %>%
     rename(Smoking_Status = sample)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Attaching package: ‘Mat

In [2]:
head(molecular_feature_df)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_070,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
NS_110,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
NS_143,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
NS_152,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
NS_187,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
NS_196,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


In [3]:
# Variable Selection to determine which variables are collinear and need to be removed

# creating correlation matrix
corr_matrix = rcorr(as.matrix(molecular_feature_df[2:3873]), type = "spearman")
# creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df = data.frame(corr_matrix$r) %>%
  rownames_to_column(var = "Variable1") %>%
  # needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  # also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.8) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  # removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

head(corr_matrix_df)

Using Variable1 as id variables



,Variable1,Variable2,Spearman Coefficient
,<chr>,<fct>,<dbl>
1,MF_586,MF_7,0.9987726
2,MF_124,MF_34,0.9984220
3,MF_516,MF_7,0.9974284
4,MF_301,MF_133,0.9973115
5,MF_54,MF_4,0.9969024
6,MF_2762,MF_974,0.9963764


In [4]:
# counting number of occurences
occurences_df = full_join(data.frame(table(corr_matrix_df$Variable1)), 
                           data.frame(table(corr_matrix_df$Variable2))) %>%
    arrange(-Freq) %>%
    # removing variables with greater than 50 occurences
    filter(Freq > 50)
occurences_df

Joining, by = c("Var1", "Freq")


Var1,Freq
<fct>,<int>
MF_4,103
MF_7,98
MF_54,84
MF_2,79
MF_93,75
MF_277,64
MF_316,57
MF_48,55
MF_516,53


In [5]:
# var selection
`%notin%` <- Negate(`%in%`)
molecular_feature_df = molecular_feature_df[ ,colnames(molecular_feature_df) %notin% occurences_df$Var1]

# first making smoking status column binary denoted by 0 = non smoker and 1 = ecig user
molecular_feature_df = molecular_feature_df %>%
    #grepl is synonymous with "contains"
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0")) 

mf_df_wo_nicotine = molecular_feature_df %>%
    #removing nicotine and cotinine
    select(-c(MF_97, MF_5))

head(molecular_feature_df)

Smoking_Status,MF_3,MF_5,MF_6,MF_11,MF_12,MF_13,MF_14,MF_15,MF_17,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,17.86943,18.02809,17.46636,18.41799,19.31361,19.46777,20.80616,18.72405,20.11458,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
0,22.58769,17.93584,17.33098,20.37485,21.12876,21.01488,19.71124,19.17696,19.82979,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
0,14.44659,16.51009,14.93820,15.72731,13.78698,15.18975,18.19551,18.84322,18.65454,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
0,23.22761,16.22547,17.59020,21.14531,22.07857,21.97502,17.30567,19.61857,18.35221,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
0,17.93876,16.73360,17.88844,17.64979,19.64269,19.32233,21.33752,19.56012,20.23552,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
0,24.36662,16.69956,16.38411,17.30008,22.80146,22.09338,19.51343,19.39161,20.72501,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


The goal of this analysis is to use molecular features [continuous] to predict smoking staus (dichotomous). This will be done using Support Vector Machine (SVM) based on a logistic regression model for classification and Random Forest.

# RF

In [6]:
rf_classification = function(dataset, outcome, pred_outcome){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) # number of trees (not sure if we wanted something different)
        p = dim(dataset)[2] - 1 # number of variables in dataset
        mtry_values = c(sqrt(p), p/2, p/3)

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))
        
        # many models have the lowest errors, so now selecting based on largest number of trees grown
        best_oob_df = rf_error_df[best_oob_errors, ]
        largest_trees = which(best_oob_df$Tree.Number == max(best_oob_df$Tree.Number))
        
        # still duplicate models w/ the lowest errors, so now selecting based on # of predictors = sqrt(p)
        best_tree_df = best_oob_df[largest_trees, ]
        default_predictor_number = which(best_tree_df$Variable.Number == max(best_tree_df$Variable.Number))


        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
                               ntree = best_tree_df$Tree.Number[default_predictor_number],
                               mtry = best_tree_df$Variable.Number[default_predictor_number])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [7]:
#calling fn
rf_values = rf_classification(molecular_feature_df, "Smoking_Status", "pred_Smoking_Status")
rf_values_wo_nicotine = rf_classification(mf_df_wo_nicotine, "Smoking_Status", "pred_Smoking_Status")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [8]:
#viewing results
rf_confusion_matrix = data.frame(Model = c(rep(c("Original RF"), times = 2)), Removed.Variables = c("None", "Nicotine & Cotinine"),
                                 rbind(rf_values[[2]], rf_values_wo_nicotine[[2]]))
rf_confusion_matrix

#viewing most significant features
head(rf_values[[3]])
head(rf_values_wo_nicotine[[3]])

Model,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original RF,None,0.8847619,0.8761905,0.8933333,0.9064286,0.8847619
Original RF,Nicotine & Cotinine,0.6619048,0.6904762,0.6333333,0.6945238,0.6619048


Predictor,MeanDecreaseGini
<chr>,<dbl>
MF_97,5.2860974
MF_5,4.1627563
MF_7221,0.3794118
MF_292,0.3086125
MF_4858,0.2381267
MF_3061,0.1980776


Predictor,MeanDecreaseGini
<chr>,<dbl>
MF_7221,0.4504978
MF_1713,0.4110509
MF_4858,0.3871062
MF_5547,0.3394656
MF_292,0.3045203
MF_3345,0.2910202


# SVM

In [9]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  #taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

In [10]:
#calling fn
svm_all = svm_classification(molecular_feature_df, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
svm_wo_nicotine = svm_classification(mf_df_wo_nicotine, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [11]:
#creating one combined df
svm_final_df = data.frame(rbind(svm_all, svm_wo_nicotine))

svm_final_df = data.frame(Model = c(rep(c("Original SVM"), times = 2)), Removed.Variables = c("None", "Nicotine & Cotinine"), svm_final_df)
svm_final_df

Model,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original SVM,None,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
Original SVM,Nicotine & Cotinine,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286


In [12]:
#all results
final_df = rbind(rf_confusion_matrix, svm_final_df)
final_df

Model,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original RF,None,0.8847619,0.8761905,0.8933333,0.9064286,0.8847619
Original RF,Nicotine & Cotinine,0.6619048,0.6904762,0.6333333,0.6945238,0.6619048
Original SVM,None,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
Original SVM,Nicotine & Cotinine,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
